In [28]:
import pandas as pd
import yfinance as yf 
from sqlalchemy import create_engine
import talib as ta
ticker_engine = create_engine('sqlite:///../../dataset/us/us_ticker_list_with_name.db')
price_engine = create_engine('sqlite:///../../dataset/us/us_ticker_seven_year_price.db')

In [29]:
name_df = pd.read_sql('TOTAL',con=ticker_engine)
name_list = name_df.Symbol.to_list()

In [30]:
# ticker_list = []
# for ticker in name_list:
#     try:
#         # datetime = total.datetime[1]
#         # datetime = str(datetime)
#         # datetime = datetime[:10]
#         df = pd.read_sql('{}'.format(ticker), con=price_engine)
#     except:
#         ticker_list.append(ticker)

In [31]:
# ticker_str = ''
# for ticker in ticker_list:
#     ticker_str = ticker_str + str(ticker) + ' '
# ticker_str

'ABR ACCO ACDI ACH ACLX ACTG ADC ADIL ADTX ADV AFRM AGAC AGMA AGMH AIMA AIR AKOA AKOB AKYA ALBO ALC ALEC ALKT ALOR ALRS ALXO AMPG AMPL AMRS ANNX AON AORT APLT APWC ARDS ARDX ARE AREC ARGX ARHS ARRW ARRY ASPA ASTE ATAX ATHA ATKR ATLO AUDC AUGX AWRE AWX AXLA AXON AYX AZ BBDO BBGI BCO BEDU BFA BFB BFRI BFS BGFV BGI BHR BIOT BNFT BNGO BNR BOAS BPT BPTH BRBR BRFH BRKA BRKB BTAI BTBT BTCM BTCS BTTX BUD BWAQ BWEN CAC CACC CBT CBTX CDEV CECE CFFI CFFS CFSB CGA CGC CGEN CHCT CHE CHEK CHK CHNR CHPT CINC CING CJJD CKPT CKX CL CLAS CLEU CLLS CLOV CLPT CLRC CLRM CLVR CRAI CRBU CRDA CRDO CRTO CRU CVEO CVLT CVLY CVNA CWENA CYN DG DGICB DKL DKS DSWL DUET EGHT EGLX ELDN EMCF EMCG EMKR ENG EOCW EPAM EPC EQD EQHA EQOS ERAS ERO ES ESTA EVGN EVGO EXPD EXPO EZPW FBP FBRT FMAO FNKO FORD FOX FRSX FRT FWRG GAMB GAMC GATE GBBK GBDC GBS GEFB GIC GLBL GOGO GOOGL GPCO GPK GRCY GRNA GRRR GRTS GTIM GTNA GVP GXII HAAC HAE HCIC HCMA HEIA HEPA HILS HIPO HMAC HOTH HP HPK HRI HSDT HTGM HTHT HUN HUSA HWEL HZO HZON IACC IC

In [27]:
# ticker_df = pd.read_sql('TOTAL',con=engine)
# ticker_list = ticker_df.Symbol.to_list()
# ticker_str = ''

# # download data    
ticker_list = ['AAPL','BABA','MSFT']
ticker_str = ''
for ticker in ticker_list:
    ticker_str = ticker_str + str(ticker) + ' '
ticker_str
data = yf.download(
    tickers = ticker_str,
    period = '7y',
    interval = '1d',
    group_by = 'ticker',
    auto_adjust = True,
    treads = True
)

# # 关闭警告的设置方法
pd.set_option('mode.chained_assignment', None)
# error_list = []
price_engine = create_engine('sqlite:///../../dataset/us/us_ticker_seven_year_price.db')
for ticker in ticker_list:
    try:
        df = data['{}'.format(ticker)]
        df.dropna(inplace=True)
        if df.empty == True:
            ticker_list.remove(ticker)
            continue
        else:
            df['datetime'] = df.index
            df['datetime'] = df.datetime.apply(lambda x: str(x)[:10])
            df = df[::-1]
            df.to_sql('{}'.format(ticker),if_exists='replace',index=None,con=price_engine)
            print('{} ok'.format(ticker))
    except:
        continue



[*********************100%***********************]  882 of 882 completed

350 Failed downloads:
- GEFB: No data found, symbol may be delisted
- LBPS: No data found, symbol may be delisted
- DUET: No data found for this date range, symbol may be delisted
- MTN: No data found for this date range, symbol may be delisted
- INOD: No data found for this date range, symbol may be delisted
- CYN: No data found for this date range, symbol may be delisted
- EPAM: No data found for this date range, symbol may be delisted
- NSSC: No data found for this date range, symbol may be delisted
- GBDC: No data found for this date range, symbol may be delisted
- KTRA: No data found for this date range, symbol may be delisted
- WGO: No data found for this date range, symbol may be delisted
- ESTA: No data found for this date range, symbol may be delisted
- SLGC: No data found for this date range, symbol may be delisted
- GPCO: No data found for this date range, symbol may be delisted
- BGFV: No data found f

In [ ]:
datetime = total.iloc[1].datetime
total.to_sql('{}'.format(datetime),if_exists='replace', con=engine, index=None)

6121

In [3]:
total = pd.DataFrame()
df = pd.DataFrame([[1,2,3,4,5]],columns=[1,2,3,5,6])
total = pd.concat([total,df])
total

,1,2,3,5,6
0,1,2,3,4,5


In [2]:
ticker_list = ['AAPL','MSFT','BABA']
ticker_str = 'AAPL MSFT BABA' 

In [18]:
total

In [7]:
try:
    engine = create_engine('sqlite:///../../dataset/us/us_ticker_with_indicator.db')
    # datetime = total.datetime[1]
    # datetime = str(datetime)
    # datetime = datetime[:10]
    total.to_sql('{}'.format(end_date),if_exists="replace",index=None, con=engine)
except:
    print('error occur')